In [1]:
from pynq import allocate
from pynq import Overlay

from enum import IntEnum
import numpy as np
import time

overlay = Overlay('./bram_test.bit')

/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/pl_server/xrt_device.py:59: UserWarning: xbutil failed to run - unable to determine XRT version
  warnings.warn("xbutil failed to run - unable to determine XRT version")


In [2]:
cdma = overlay.axi_cdma_0

In [3]:
Overlay?

In [4]:
input_buffer0 = allocate(shape=(2048,), dtype=np.uint32)
input_buffer1 = allocate(shape=(2048,), dtype=np.uint32)
input_buffer2 = allocate(shape=(2048,), dtype=np.uint32)
input_buffer3 = allocate(shape=(2048,), dtype=np.uint32)

output_buffer0 = allocate(shape=(2048,), dtype=np.uint32)
output_buffer1 = allocate(shape=(2048,), dtype=np.uint32)
output_buffer2 = allocate(shape=(2048,), dtype=np.uint32)
output_buffer3 = allocate(shape=(2048,), dtype=np.uint32)

for i in range(2048):
   input_buffer0[i] = i
   input_buffer1[i] = i+2048
   input_buffer2[i] = i+2048*2
   input_buffer3[i] = i+2048*3

print("Input:")

print()

def getbit(value, order):
    orderVal = 2**order
    tmpbin = value & orderVal
    if (tmpbin):
        return 1
    else:
        return 0

def changebit(value, order, bit):
    if (bit == 1):
        return value | (1 << order)
    else:
        return value & ~(1 << order)

class Cdma(IntEnum):
    CDMACR = 0x0
    CDMASR = 0x4
    SA = 0x18
    DA = 0x20
    BTT = 0x28

def transfer(cdma, src, dst, size):
    # Step 1
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    if (cdmasrIdle != 1):
        print("CDMA is busy..")
        return

    # Step 2
    cdmacr = cdma.read(Cdma.CDMACR)
    cdmacr = changebit(cdmacr, 12, 1) # set IOC_IrqEn
    cdmacr = changebit(cdmacr, 14, 1) # set ERR_IrqEn
    cdma.write(Cdma.CDMACR, cdmacr)

    # Step 3
    cdma.write(Cdma.SA, src)

    # Step 4
    cdma.write(Cdma.DA, dst)

    # Step 5
    cdma.write(Cdma.BTT, size)

    # Step 6
    print("Transferring...")
    cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    while (cdmasrIdle != 1):
        print(".", end="")
        cdma.read(Cdma.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)

    # Step 7-8
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasr = changebit(cdmasr, 12, 1) # clear IOC_Irq
    cdma.write(Cdma.CDMASR, cdmasr)

    print("Transfered " + str(size) + " bytes from " + str(src) + " to " + str(dst))
    print("CDMA Done.")

def reset(cdma):
    cdmacr = cdma.read(Cdma.CDMACR)
    print(type(cdmacr))
    print(cdmacr)
    cdmacr = changebit(cdmacr, 2, 1)
    print(cdmacr)
    cdma.write(Cdma.CDMACR, cdmacr)

print("==== Test CDMA ====")

start_time = time.time()
transfer(cdma, input_buffer0.physical_address, 0xE000_0000, 2048*4)
transfer(cdma, 0xE000_0000, output_buffer0.physical_address, 2048*4)

transfer(cdma, input_buffer1.physical_address, 0xE200_0000, 2048*4)
transfer(cdma, 0xE200_0000, output_buffer1.physical_address, 2048*4)

transfer(cdma, input_buffer2.physical_address, 0xE400_0000, 2048*4)
transfer(cdma, 0xE400_0000, output_buffer2.physical_address, 2048*4)

transfer(cdma, input_buffer3.physical_address, 0xE600_0000, 2048*4)
transfer(cdma, 0xE600_0000, output_buffer3.physical_address, 2048*4)


end_time = time.time()

print("%s seconds" % (end_time - start_time))
print()

print(output_buffer0)
print(output_buffer1)
print(output_buffer2)
print(output_buffer3)



Input:

==== Test CDMA ====
Transferring...
Transfered 8192 bytes from 965214208 to 3758096384
CDMA Done.
Transferring...
Transfered 8192 bytes from 3758096384 to 965246976
CDMA Done.
Transferring...
Transfered 8192 bytes from 965222400 to 3791650816
CDMA Done.
Transferring...
Transfered 8192 bytes from 3791650816 to 965255168
CDMA Done.
Transferring...
Transfered 8192 bytes from 965230592 to 3825205248
CDMA Done.
Transferring...
Transfered 8192 bytes from 3825205248 to 965263360
CDMA Done.
Transferring...
Transfered 8192 bytes from 965238784 to 3858759680
CDMA Done.
Transferring...
Transfered 8192 bytes from 3858759680 to 965271552
CDMA Done.
0.012778997421264648 seconds

[   0    1    2 ... 2045 2046 2047]
[2048 2049 2050 ... 4093 4094 4095]
[4096 4097 4098 ... 6141 6142 6143]
[6144 6145 6146 ... 8189 8190 8191]
